In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType
from pyspark.sql.functions import col, concat_ws, lit, regexp_replace

# Create a Spark session
spark = SparkSession.builder.appName("spark").getOrCreate()

# Define the schema with ArrayType for topicTags
schema = StructType([
    StructField("difficulty", StringType(), True),
    StructField("freqBar", DoubleType(), True),
    StructField("status", StringType(), True),
    StructField("title", StringType(), True),
    StructField("link", StringType(), True),
    StructField("topicTag1", StringType(), True),  # Read as StringType initially
    StructField("topicTag2", StringType(), True),
    StructField("topicTag3", StringType(), True),
    StructField("topicTag4", StringType(), True),
    StructField("topicTag5", StringType(), True),
    StructField("topicTag6", StringType(), True),
    StructField("topicTag7", StringType(), True),
    StructField("topicTag8", StringType(), True),
    StructField("topicTag9", StringType(), True),
    StructField("topicTag10", StringType(), True),
    StructField("topicTag11", StringType(), True)  # Read as StringType initially
])

# Function to read CSV and convert topicTags from JSON
def read_csv_with_array_conversion(file_path, company_name):
    df = spark.read.csv(file_path, header=True, schema=schema, escape='"')
    df = df.withColumn("topicTags", concat_ws(",", col("topicTag1"), col("topicTag2"), col("topicTag3"), col("topicTag4"), col("topicTag5"), col("topicTag6"), col("topicTag7"), col("topicTag8"), col("topicTag9"), col("topicTag10"), col("topicTag11")))
    # Clean up the topicTags column and remove " and [ ] characters
    df = df.withColumn("topicTags", regexp_replace(col("topicTags"), "[\"\\[\\]]", ""))\
        .drop("topicTag1", "topicTag2", "topicTag3", "topicTag4", "topicTag5", "topicTag6", "topicTag7", "topicTag8", "topicTag9", "topicTag10", "topicTag11")
    df = df.withColumn("company", lit(company_name))
    return df

# Load the data from the CSV files
amazon = read_csv_with_array_conversion("../amazon.csv", "Amazon")
microsoft = read_csv_with_array_conversion("../microsoft.csv", "Microsoft")
meta = read_csv_with_array_conversion("../Meta.csv", "Meta")
uber = read_csv_with_array_conversion("../Uber.csv", "Uber")
google = read_csv_with_array_conversion("../Google.csv", "Google")

# Show the DataFrames
amazon.show(5, False)
microsoft.show(5, False)
meta.show(5, False)
uber.show(5, False)
google.show(5, False)

24/08/06 16:31:58 WARN Utils: Your hostname, BMMDVADHIKRAMM.local resolves to a loopback address: 127.0.0.1; using 172.16.83.178 instead (on interface en0)
24/08/06 16:31:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/06 16:31:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/06 16:32:03 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 16
CSV file: file:///Users/adhikram.m/personal/Study_Resources/amazon.csv


+----------+-----------------+------+----------------------------------------------+-----------------------------------------------------------------------------------------+---------------------------------------+-------+
|difficulty|freqBar          |status|title                                         |link                                                                                     |topicTags                              |company|
+----------+-----------------+------+----------------------------------------------+-----------------------------------------------------------------------------------------+---------------------------------------+-------+
|Easy      |83.39821340776916|ac    |Two Sum                                       |https://leetcode.com/problems/two-sum/description/                                       |Array,Hash Table                       |Amazon |
|Medium    |64.16891209031364|ac    |Add Two Numbers                               |https://leetcode.com/pro

24/08/06 16:32:03 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 16
CSV file: file:///Users/adhikram.m/personal/Study_Resources/microsoft.csv
24/08/06 16:32:03 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 16
CSV file: file:///Users/adhikram.m/personal/Study_Resources/Meta.csv
24/08/06 16:32:04 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 16
CSV file: file:///Users/adhikram.m/personal/Study_Resources/Uber.csv
24/08/06 16:32:04 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 16
CSV file: file:///Users/adhikram.m/personal/Study_Resources/Google.csv


In [5]:
from pyspark.sql.functions import collect_list, concat_ws, count

# Union all the DataFrames
unionData = amazon.union(microsoft).union(meta).union(uber).union(google)

# Perform the required transformations and aggregations
finalData = unionData \
    .withColumn("companyFreq", concat_ws("->", unionData.company, unionData.freqBar)) \
    .groupby("difficulty", "status", "title", "link", "topicTags") \
    .agg(
        collect_list("companyFreq").alias("companyFreq"),
        count("company").alias("companyCount"),
    ).withColumn("topicTags", concat_ws(", ", unionData.topicTags))\
    .withColumn("companyFreq", concat_ws(", ", "companyFreq"))

# Show the final DataFrame
finalData.show(5, False)
finalData.write.csv("Aggregated", header=True)

24/08/06 16:33:28 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 16
CSV file: file:///Users/adhikram.m/personal/Study_Resources/Google.csv
24/08/06 16:33:28 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 16
CSV file: file:///Users/adhikram.m/personal/Study_Resources/Meta.csv
24/08/06 16:33:28 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 16
CSV file: file:///Users/adhikram.m/personal/Study_Resources/amazon.csv
24/08/06 16:33:28 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 16
CSV file: file:///Users/adhikram.m/personal/Study_Resources/Uber.csv
24/08/06 16:33:28 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fi

+----------+------+--------------------------------------------------------+---------------------------------------------------------------------------------------------------+-----------------------------+------------------------------------------------------------------------------------------------------------------------------------+------------+
|difficulty|status|title                                                   |link                                                                                               |topicTags                    |companyFreq                                                                                                                         |companyCount|
+----------+------+--------------------------------------------------------+---------------------------------------------------------------------------------------------------+-----------------------------+----------------------------------------------------------------------------------------

24/08/06 16:33:28 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 16
CSV file: file:///Users/adhikram.m/personal/Study_Resources/Google.csv
24/08/06 16:33:28 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 16
CSV file: file:///Users/adhikram.m/personal/Study_Resources/Meta.csv
24/08/06 16:33:28 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 16
CSV file: file:///Users/adhikram.m/personal/Study_Resources/amazon.csv
24/08/06 16:33:28 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 16
CSV file: file:///Users/adhikram.m/personal/Study_Resources/microsoft.csv
24/08/06 16:33:28 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number 